In [2]:
import os
import json
import glob
import cv2
from PIL import Image
from PIL.ExifTags import TAGS

In [10]:
# Directory paths
dataset_dir = './dataset'
images_data = glob.glob(os.path.join(dataset_dir, 'images', '*.jpg'))
labels_data = {}
result_dir = os.path.join(dataset_dir, 'result')

for each_json in glob.glob(os.path.join(dataset_dir, 'labels', '*.json')):
    json_name = each_json.split('\\')[2]
    json_name = json_name.split('.')[0]
    labels_data[json_name] = each_json
json_used = len(labels_data)

In [13]:
## 위험도를 예측할 수 있는 데이터로 가공하는 코드
import json
import os
import glob
import pandas as pd
from pandas import DataFrame

import statistics

dataset_dir = './dataset'
labels_data = {}
box_field_names = ["id", # 아이디
                   "image_width",  # 이미지 너비
                   "image_height", # 이미지 높이
                   "box_count",  # 박스 갯수
                   "box_size_avg",  #박스 평균 사이즈
                   "box_dist_avg",  #박스간 거리
                   "box_normalized_count", # 표준화된 박스갯수, 평균값에 가까운 박스들 (50%~150%)
                  "box_normalized_count (above average)", #평균보다 1.5배 큰 박스들
                  "box_normalized_count (below average)", #평균보다 0.5배 작은 박스들
                  "box_normalized_size_avg", #  표준화된 박스갯수들로 계산한 박스크기
                  "box_normalized_dist_avg", # 표준화된 박스갯수들로 계산한 박스간 거리
                  "normalization_warp_ratio", # 이미지의 왜곡도 수준 - 수식: warp_ratio = box_count / box_normalized_count
                  "box_normalized_plus_dist_avg", # 라벨값을 보정한 뒤에 계산한 박스간 거리
                   "d_box_norm_above_size", # 평균보다 작은 박스들의 평균사이즈
                  "d_box_norm_below_size", # 평균보다 큰 박스들의 평균사이즈
                  "d_box_normalized_points_far_distance", # 평균적으로 큰 박스들과 평균적으로 작은 박스들간의 거리
                   "box_most_large_x", #가장 큰 박스의 x
                   "box_most_large_y", #가장 큰 박스의 y
                   "box_most_small_x", #가장 작은 박스의 x
                   "box_most_small_y"  # 가장 작은 박스의 y
                  ] 
box_data_total = []

for each_json in glob.glob(os.path.join(dataset_dir, 'labels', '*.json')):
    json_name = each_json.split('\\')[2]
    json_name = json_name.split('.')[0]
    labels_data[json_name] = each_json

for each_json in labels_data:
    current_num = each_json
    if int(current_num) % 100 == 1:
        print("current:",current_num)

    # Load JSON data
    try:
        with open(labels_data[current_num], 'r') as f:
            json_data = f.read()
        data = json.loads(json_data)
    except:
        print("error!:",current_num,"is not lodable")
        continue
    
    image = Image.open(os.path.join('./dataset', 'images', current_num+'.jpg'))
    image_width = image.width
    image_height = image.height
    
    # Parse JSON data
    data = json.loads(json_data)
    boxes = data['boxes'].copy()
    points = data['points'].copy()
    
    d_box_count = 0
    d_box_size = 0
    d_box_dist_avg = 0
    temp_box_size_x = 0
    temp_box_size_y = 0
    
    box_most_large_x = 0
    box_most_large_y = 0
    box_most_small_x = 9999
    box_most_small_y = 9999

    for each_box in boxes:
        x1, y1, x2, y2 = each_box
        d_box_count += 1
        temp_box_size_x += x2-x1
        temp_box_size_y += y2-y1
        
        if box_most_large_y < y2-y1: #세로를 기준으로
            box_most_large_x = x2-x1
            box_most_large_y = y2-y1
        if box_most_small_y > y2-y1: #세로를 기준으로
            box_most_small_x = x2-x1
            box_most_small_y = y2-y1
            
    
    if d_box_count > 0:
        temp_box_size_x = temp_box_size_x/d_box_count
        temp_box_size_y = temp_box_size_y/d_box_count
        d_box_size = max(temp_box_size_x, temp_box_size_y) # 정사각형으로 간주
    
    temp_box_distances = []
    for __q in range(len(points)):
        minimum = 999999
        for __w in range(len(points)):
            if __q == __w:
                continue
            center1 = (points[__q][0]+points[__q][1])/2
            center2 = (points[__w][0]+points[__w][1])/2
            result = abs(center1-center2)
            if result < minimum:
                minimum = result
        if minimum == 0: # 원래는 0이 나오면 안되는데 가끔 나옴. 라벨 문제로 보임.
            continue
        temp_box_distances.append(minimum)
    
    if len(temp_box_distances) > 0:
        d_box_dist_avg = statistics.mean(temp_box_distances)
        
    ##------------------------------------
    d_box_count_normalized = 0
    d_box_count_above_middle = 0
    d_box_count_below_middle = 0
    d_box_size_normalized = 0
    d_box_normalized_dist_avg = 0
    temp_box_size_x = 0 # 재계산
    temp_box_size_y = 0 # 재계산
    skip_count = -1
    
    box_above_list = [] 
    box_below_list = []
    point_above_list = []
    point_below_list = []
    pop_targets = []
    temp_box_size_x = 0
    temp_box_size_y = 0
    for each_box in boxes: ## 다시 박스를 확인해서 평균에서 벗어나는지 확인
        skip_count += 1
        x1, y1, x2, y2 = each_box
        y_size_check = y2-y1
        if y_size_check > d_box_size*1.5:
            d_box_count_above_middle += 1
            box_above_list.append(boxes[skip_count])
            #point_above_list.append(points.pop(skip_count))
            skip_count -= 1
            continue
        elif y_size_check < d_box_size*0.5:
            d_box_count_below_middle += 1
            #box_below_list.append(boxes.pop(skip_count))
            box_below_list.append(boxes[skip_count])
            #point_below_list.append(points.pop(skip_count))
            skip_count -= 1
            continue
        d_box_count_normalized += 1
        temp_box_size_x += x2-x1
        temp_box_size_y += y2-y1
    if d_box_count_normalized > 0:
        temp_box_size_x = temp_box_size_x/d_box_count_normalized
        temp_box_size_y = temp_box_size_y/d_box_count_normalized
        d_box_size_normalized = max(temp_box_size_x, temp_box_size_y) # 정사각형으로 간주
    
    temp_box_size_x = 0
    temp_box_size_y = 0
    if d_box_size_normalized == 0:
        for each_box in box_above_list: ## 다시 박스를 확인해서 평균에서 벗어나는지 확인
            skip_count += 1
            x1, y1, x2, y2 = each_box
            d_box_count_normalized += 1
            temp_box_size_x += x2-x1
            temp_box_size_y += y2-y1
        if d_box_count_normalized > 0:
            temp_box_size_x = temp_box_size_x/d_box_count_normalized
            temp_box_size_y = temp_box_size_y/d_box_count_normalized
            d_box_size_normalized = max(temp_box_size_x, temp_box_size_y) # 정사각형으로 간주
        d_box_count_above_middle = 0
        
    # 포인트 부분 다시
    temp_box_distances = []
    for __q in range(len(points)):
        minimum = 999999
        for __w in range(len(points)):
            if __q == __w:
                continue
            center1 = (points[__q][0]+points[__q][1])/2
            center2 = (points[__w][0]+points[__w][1])/2
            result = abs(center1-center2)
            if result < minimum:
                minimum = result
        if minimum == 0: # 원래는 0이 나오면 안되는데 가끔 나옴. 라벨 문제로 보임.
            continue
        temp_box_distances.append(minimum)
    
    d_box_normalized_dist_avg = 0
    if len(temp_box_distances) > 0:
        d_box_normalized_dist_avg = statistics.mean(temp_box_distances)
    
    d_data_warp_ratio = 0
    if d_box_count > 0: # 왜곡의 수준
        d_data_warp_ratio = 1-(d_box_count_normalized/d_box_count)
        
    
        val_portion_to_adjust = 100 / d_box_size_normalized
    
    # 다시 또 재계산
    #boxes = data['boxes']
    #points = data['points'] # 새로 할 필요 없을듯
    #new_boxes = []
    new_points = []
    
    # 포인트 부분 재계산
    for each_point in points:
        px, py = each_point
        y_rate = py/image_height
        new_y = py*y_rate*val_portion_to_adjust
        new_x = px*y_rate*val_portion_to_adjust
        new_points.append([new_x, new_y])
        
    # 포인트 부분 다시 (3차)
    temp_box_distances = []
    d_box_normalized_plus_dist_avg = 0
    points = new_points
    for __q in range(len(points)):
        minimum = 999999
        for __w in range(len(points)):
            if __q == __w:
                continue
            center1 = (points[__q][0]+points[__q][1])/2
            center2 = (points[__w][0]+points[__w][1])/2
            result = abs(center1-center2)
            if result < minimum:
                minimum = result
        if minimum == 0: # 원래는 0이 나오면 안되는데 가끔 나옴. 라벨 문제로 보임.
            continue
        temp_box_distances.append(minimum)
    
    d_box_normalized_plus_dist_avg = 0
    if len(temp_box_distances) > 0:
        d_box_normalized_plus_dist_avg = statistics.mean(temp_box_distances)
    
    
    d_box_norm_above_size = 0
    d_box_norm_below_size = 0
    d_box_normalized_points_far_distance = 0
    
    if d_box_count > 0:
        # 박스 사이즈 계산을 평균이상이하의 평균을 계산
        temp_box_size_x = 0
        temp_box_size_y = 0
        ## 이하
        for each_box in box_below_list:
            x1, y1, x2, y2 = each_box
            d_box_count += 1
            temp_box_size_x += x2-x1
            temp_box_size_y += y2-y1

        if d_box_count > 0:
            temp_box_size_x = temp_box_size_x/d_box_count
            temp_box_size_y = temp_box_size_y/d_box_count
            d_box_norm_below_size = max(temp_box_size_x, temp_box_size_y) # 정사각형으로 간주
        
        temp_box_size_x = 0
        temp_box_size_y = 0
        ## 이상
        for each_box in box_above_list:
            x1, y1, x2, y2 = each_box
            d_box_count += 1
            temp_box_size_x += x2-x1
            temp_box_size_y += y2-y1

        if d_box_count > 0:
            temp_box_size_x = temp_box_size_x/d_box_count
            temp_box_size_y = temp_box_size_y/d_box_count
            d_box_norm_above_size = max(temp_box_size_x, temp_box_size_y) # 정사각형으로 간주
                   
        
        # 포인트 다시 구하기
        skip_count = -1
        for each_box in boxes: ## 다시 박스를 확인해서 평균에서 벗어나는지 확인
            skip_count += 1
            x1, y1, x2, y2 = each_box
            y_size_check = y2-y1
            if y_size_check > d_box_size*1.5:
                point_above_list.append(points.pop(skip_count))
                skip_count -= 1
                continue
            elif y_size_check < d_box_size*0.5:
                point_below_list.append(points.pop(skip_count))
                skip_count -= 1
                continue
        # 포인트, 가장 작은 박스랑 큰 박스랑 비교
        if len(point_above_list) > 0 and len(point_below_list) > 0:
            temp_box_distances = []
            for __q in range(len(point_above_list)):
                maximum = 0
                for __w in range(len(point_below_list)):
                    if __q == __w:
                        continue
                    center1 = (point_above_list[__q][0]+point_above_list[__q][1])/2
                    center2 = (point_below_list[__w][0]+point_below_list[__w][1])/2
                    result = abs(center1-center2)
                    if result > maximum:
                        maximum = result
                temp_box_distances.append(maximum)

            if len(temp_box_distances) > 0:
                d_box_normalized_points_far_distance = statistics.mean(temp_box_distances)
        
    box_data = []
    box_data.append(current_num)
    box_data.append(image_width)
    box_data.append(image_height)
    box_data.append(d_box_count)
    box_data.append(d_box_size)
    box_data.append(d_box_dist_avg)
    box_data.append(d_box_count_normalized)
    box_data.append(d_box_count_above_middle)
    box_data.append(d_box_count_below_middle)
    box_data.append(d_box_size_normalized)
    box_data.append(d_box_normalized_dist_avg)
    box_data.append(d_data_warp_ratio)
    box_data.append(d_box_normalized_plus_dist_avg)
    box_data.append(d_box_norm_above_size)
    box_data.append(d_box_norm_below_size)
    box_data.append(d_box_normalized_points_far_distance)
    box_data.append(box_most_large_x)
    box_data.append(box_most_large_y)
    if box_most_small_x == 9999:
        box_most_small_x = 0
    if box_most_small_y == 9999:
        box_most_small_y = 0
    box_data.append(box_most_small_x)
    box_data.append(box_most_small_y)
    
    box_data_total.append(box_data)

df_data = DataFrame(box_data_total, columns=box_field_names)
df_data.to_csv(os.path.join('./', 'refined_box_data.csv'), index=False)
print("finished")

current: 0001
finished


# 추가 코드
이건 필요할때만 실행하면 됩니다.

In [11]:
### 박스 결과를 입혀서 추출하는 코드
### 굳이 실행 안해도 되는 코드임.
for img_count in range(len(images_data)):
    image_loc = images_data[img_count]
    image_name = images_data[img_count].split('\\')[2]
    assoc_key = image_name.split('.')[0]

    # Load JSON data
    try:
        with open(labels_data[assoc_key], 'r') as f:
            json_data = f.read()
        data = json.loads(json_data)
    except:
        print("error!:",img_count,assoc_key)
        continue


    # Parse JSON data

    # Load image
    img = cv2.imread(image_loc)
 
    # Get bounding box coordinates
    boxes = data['boxes']

    # Draw bounding boxes on image
    for box in boxes:
        x1, y1, x2, y2 = box
        x1 = int(x1)
        y1 = int(y1)
        x2 = int(x2)
        y2 = int(y2)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Save output image    
    output_file = os.path.join(result_dir, 'refined'+assoc_key+'.jpg')
    cv2.imwrite(output_file, img)


KeyboardInterrupt: 